In [159]:
import numpy as np
import pandas as pd

In [160]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [161]:
data = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
train_data = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
valid_data = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [162]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [163]:
def predict_output(feature_matrix, weights):
    prediction=np.dot(feature_matrix,weights)
    return(prediction)

In [164]:
np.sqrt(np.sum(data['sqft_living'].apply(lambda x : x**2)))

334257.26412301051

In [165]:
np.linalg.norm(data['sqft_living'], axis=0)

334257.26412301051

In [166]:
def normalize_features(features):
    norms=np.linalg.norm(features, axis=0)
    normalized_features=features/norms
    return (normalized_features, norms)

In [329]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    prediction=predict_output(feature_matrix, weights)
    ro=np.sum(feature_matrix[:,i]*(output-prediction + weights[i]*feature_matrix[:,i]))
    if i == 0: # intercept -- do not regularize
        weights[i] = ro
    elif ro < -l1_penalty/2.:
        weights[i]=ro+(l1_penalty/2.)
    elif ro > l1_penalty/2.:
        weights[i] = ro-(l1_penalty/2.)
    else:
        weights[i] = 0.
    
    return weights[i]

In [351]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    converge = False
    weights = initial_weights
    start_weight = np.array(initial_weights)
    change = np.zeros(initial_weights.shape)
    while not converge:
        for i in range(len(weights)):
            weight = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change[i]= np.absolute(weights[i] -weight)
            weights[i]=weight
        print weights,change
        if ( max(change) <= tolerance):
            converge = True
    return (weights)

In [345]:
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


In [316]:
feature_1=np.array([np.ones(data.shape[0]),data['sqft_living'],data['bedrooms']])

In [317]:
feature=np.array([data['sqft_living'],data['bedrooms']])
feature=feature.transpose()

In [318]:
feature_1=feature_1.transpose()

In [319]:
normalized_feature,w=normalize_features(feature_1)

In [313]:
normalized_feature=np.append(np.ones(data.shape[0]).reshape(data.shape[0],1),normalized_feature,axis=1)

In [320]:
normalized_feature

array([[ 0.00680209,  0.00353021,  0.00583571],
       [ 0.00680209,  0.00768869,  0.00583571],
       [ 0.00680209,  0.00230361,  0.00389048],
       ..., 
       [ 0.00680209,  0.00305154,  0.00389048],
       [ 0.00680209,  0.00478673,  0.00583571],
       [ 0.00680209,  0.00305154,  0.00389048]])

In [321]:
weights=np.array([1,4,1])

In [322]:
weights.shape

(3L,)

In [332]:
prediction=predict_output(normalized_feature, weights)
ro=np.sum(normalized_feature[:,1]*(data['price']-prediction + weights[1]*normalized_feature[:,1]))

In [333]:
ro/2

43969736.340913944

In [334]:
prediction=predict_output(normalized_feature, weights)
ro=np.sum(normalized_feature[:,2]*(data['price']-prediction + weights[2]*normalized_feature[:,2]))

In [335]:
ro/2.

40483351.70269243

In [327]:
output=data['price']

In [ ]:
prediction=predict_output(feature_matrix, weights)
ro=np.sum(feature_matrix[:,i]*(output-prediction + weights[i]*feature_matrix[:,i]))
if i == 0: # intercept -- do not regularize
    weights[i] = ro
elif ro < -l1_penalty/2.:
    weights[i]=ro+(l1_penalty/2.)
elif ro > l1_penalty/2.:
    weights[i] = ro-(l1_penalty/2.)
else:
    weights[i] = 0.
    
return weights[i]

In [347]:
weights=np.array([0.,0.,0.])
l1_penalty=1e7
tolerance=1.0

In [353]:
lasso_cyclical_coordinate_descent(normalized_feature, output, weights, l1_penalty, tolerance)

[ 55468421.66091404  32197788.17236555         0.        ] [ 0.  0.  0.]


array([ 55468421.66091404,  32197788.17236555,         0.        ])

In [354]:
np.abs(-4.5)

4.5